## Data Compression Problem

Data Compression คือ การบีบอัดข้อมูลให้เล็กลง ยกตัวอย่างเช่น ในกรณีที่เราต้องการเก็บข้อมูลประเภท String ที่ประกอบด้วยตัวอักษรต่างๆ <br>
เราสามารถกำหนดขนาดของข้อมูลได้ทั้งหมด 2 รูปแบบ ได้แก่
1. <span style="font-weight: bold;">ใช้ ASCII Table</span>
หมายความว่า ทุกๆ ตัวอักษรจะใช้ขนาด 8 bits เสมอในการเก็บ ทำให้ปัญหาคือ <span style="color: yellow;">ข้อมูลที่เก็บมีขนาดที่ใหญ่ แม้ตัวอักษรจะน้อยก็ตาม</span>

<img src="https://www.sciencebuddies.org/cdn/references/ascii-table.png" style="width:600px;">


2. <span style="font-weight: bold;">การกำหนดขนาดขึ้นมาเอง (Own fixed-length size codes)</span>>
หมายความว่า เราดูตัวอักษรที่อยู่ใน String ที่เราจะเก็บก่อน แล้วดู Unique items  <br> จากนั้นเรากำหนดรหัสแทนแต่ละตัวอักษรขึ้นมาเอง เช่น ถ้าหากมี A-G
เราก็จะกำหนดเป็น 3 bits (แทน A = 000 จนถึง G = 110)

<span style="color: yellow;">แต่ปัญหาคือ</span> ถ้าหากเรามีตัวอักษรที่เพิ่มมาอีกไม่กี่ตัว แล้วมันทำให้ทุกตัวเพิ่มรหัสเป็น 4-bit แทน
มันจะเปลืองเนื้อที่มาก 

3. <span style="font-weight: bold;">Variable-length encoding</span>

คือการแปลงให้เป็นรหัสที่มีขนาดไม่เท่ากัน โดยอาศัยการแปลงจากอักษรเป็นโค้ด และเข้าถึงการแปลโดย Binary Tree <br> 
ที่มีหลักการคือโหนดลูกซ้ายเป็น 0 โหนดลูกขวาเป็น 1 
อีกหลักการคือต้องเป็น Prefix Free code ซึ่งหมายถึง ห้ามมีเลขที่ตัวหน้าซ้ำกัน ยกตัวอย่างเช่น หากมี 01 เป็น A แล้ว <br>
จะไม่สามารถมี 011 ได้ เพราะมันจะถูกแปลเป็น A ดังนั้น Prefix ตัวก่อนหน้านั้นห้ามซ้ำกันเลย

ต่อมา Samuel Morse ได้เสนอให้แนวคิดดังกล่าว มีการหาความถี่ของตัวที่ใช้บ่อยก่อน (หา Frquency ของ Unique items) <br>
ถ้าเราใช้บ่อยจะทำให้ขนาดข้อมูลของรหัสเราสั้นในการแทนรหัส <br>

Fun fact : รหัส Morse ที่คิดค้นขึ้นก็ใช้หลักการความถี่ตัวอักษรที่ใช้บ่อย เช่น สระ AEIOU จะเป็นรหัสที่สั้นกว่า

จากแนวคิดทั้งหมดของ Variable-length encoding เราจะเรียกว่า <span style="color: orange; font-weight:bold;">Huffman's algorithm</span>

## Practice II : Huffman's Algorithm

### หลักการการทำ Huffman's Algorithm

ในอัลกอริทึมนี้จะมีต้นไม้เพื่อเข้าถึงรหัส ซึ่งเรียกว่า <span style="color: hotpink;">"Huffman tree"</span> ส่วนตัวโค้ดจะเรียกว่า <span style="color:lime">"Huffman code"</span>

Step 1 : หาความถี่ของแต่ละตัวอักษร จากนั้นสร้าง Tree ที่มีโหนดเดี่ยวจำนวน n ต้น โดยให้กำหนดค่าความถี่ของโหนดและ
ทำสัญลักษณ์ว่าตัวอักษรอะไร

Step 2 : ทำการวนซ้ำ จับ 2 trees ที่มี weight น้อยที่สุดในแต่ละรอบมาประกอบเป็น Tree ใหม่ โดยสร้างเป็น <br>
Tree ที่มีโหนดลูกซ้ายขวาเป็นตัวที่เราจับมา และนำค่า Weight มารวมกันเป็นโหนดพ่อแม่ จากนั้นมันจะนับเป็น Tree ตัวใหม่
จากนั้นวนซ้ำทำไปเรื่อยๆ จนกว่าจะไม่เหลือต้นไม้ให้จับคู่กัน

<img src="https://cgi.luddy.indiana.edu/~yye/c343-2019/images/Huffman-tree-Fig5.24.png">

In [ ]:
# Huffman's Tree

class NodeTree():
    def __init__(self, freq, char, left=None, right=None):
        self.left = left
        self.right = right
        self.freq = freq
        self.char = char

    def setCode(self, dir):
        self.code = dir

    def children(self):
        return (self.left, self.right)
    
    def __str__(self):
        return '%s_%s' % (self.left, self.right)
    
    def __lt__(self, next):
        return self.freq < next.freq
    
# เอาไว้หาความถี่จากตัวอักษรใน string
def CalculateFrequency(string):
    freq_list = {}
    for char in string:
        if char not in freq_list:
            freq_list[char] = 1
        else:
            freq_list[char] += 1
    return freq_list


def HuffmanEncode(string):
    # 1. หาความถี่ของตัวอักษรใน string
    freq_list = CalculateFrequency(string)  
    characters, freqs = freq_list.keys() , freq_list.values()
    print("Characters : ", characters)
    print("Frequencies : ", freqs)
      
    nodes = []
      
    # สร้าง node จากความถี่ของตัวอักษร แล้วเก็บไว้เพื่อจับคู่
    for char in characters:  
        currNode = NodeTree(freq_list[char], char)
        nodes.append(currNode)
      
    while len(nodes) > 1:  
        
        # ทำการเรียงความถี่จากน้อยไปมาก
        sorted_nodes = sorted(nodes, key = lambda x: x.freq)  

        # จับคู่ 2 ตัวที่น้อยที่สุด
        right = sorted_nodes[0]
        left = sorted_nodes[1]
      
        # กำหนด code ให้ node ที่มากกว่า
        left.setCode(0)
        right.setCode(1) 
      
        # สร้าง node ใหม่ โดยให้ความถี่เท่ากับผลรวมของ node ทั้ง 2 ตัวที่น้อยที่สุด 
        newNode = NodeTree(left.freq + right.freq , left.char + right.char , left, right)  
      
        # ลบ node ที่เก่าออก แล้วใส่โหนดใหม่เข้าไปแทน เพื่อจับคู่ต่อไป
        nodes.remove(left)
        nodes.remove(right)
        nodes.append(newNode)
              
    huffmanEncoding = CalculateCodes(the_nodes[0])  
    print("symbols with codes", huffmanEncoding)  
    TotalGain(the_data, huffmanEncoding)  
    encoded_output = OutputEncoded(the_data,huffmanEncoding)  
    return encoded_output, the_nodes[0]  
        